# Setting Up the Transfer Functions

In [1]:
%load_ext autoreload

## Notebook Initialization

In [2]:
%autoreload
import numpy as np
import pickle
from tqdm import tqdm_notebook as tqdm

from darkhistory.spec.spectrum import Spectrum
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
import darkhistory.physics as phys
import darkhistory.utilities as utils

np.set_printoptions(threshold=np.nan)

## Import Raw Data

In [3]:
file_name = "/Users/hongwan/Dropbox (MIT)/Photon Deposition/transferfunction_withloweng_nointerp.npy"
raw_tf = np.load(file_name)

## Raw Data Manipulation

The raw data is a large array with no abscissa values included. We need to create the abscissae related to this large array (the abscissa of photon energies and electron energies). 

First, we swap some of the axes. The initial array has dimensions corresponding to (injected photon energy, redshift, $x_e$, outgoing energy, type), where type is high-energy photons, low-energy photons and low-energy electrons respectively. 

In [4]:
raw_tf = np.swapaxes(raw_tf, 0, 1)
raw_tf = np.swapaxes(raw_tf, 1, 2)
raw_tf = np.flip(raw_tf, axis=0)

At this point, the order is now (redshift, $x_e$, input photon energy, output energy, type), and the order is in decreasing redshift.

The abscissa for the transfer functions are as follows. For the injected photon energy and redshifts,

In [5]:
# Input energy abscissa. 
in_eng_step = 500
in_eng_low = 3e3 + 100.
in_eng_upp = 5e3 * np.exp(39 * np.log(1e13/5e3) / 40)
in_eng_arr = (
    in_eng_low * np.exp((np.arange(in_eng_step)) * 
              np.log(in_eng_upp/in_eng_low) / in_eng_step)
)

# Redshift abscissa, decreasing order. 
rs_step = 50
rs_upp  = 31
rs_low  = 4

log_rs = (
    np.log(rs_low) + (np.arange(rs_step) + 1)
    *(np.log(rs_upp) - np.log(rs_low))/rs_step
)

log_rs_arr = np.flipud(log_rs)

The output abscissa depends on the injection energy, and are different for photons and electrons. We write these as functions of the injection energy for convenience.

In [6]:
def get_out_photeng(in_eng):
    
    log_bin_width = np.log((phys.me + in_eng)/1e-4)/500
    bin_boundary  = 1e-4 * np.exp(np.arange(501) * log_bin_width)
    bin_boundary_low = bin_boundary[0:500]
    bin_boundary_upp = bin_boundary[1:501]

    return np.sqrt(bin_boundary_low * bin_boundary_upp)

def get_out_eleceng(in_eng):
    
    log_bin_width = np.log(in_eng)/500
    bin_boundary  = phys.me + np.exp(np.arange(501) * log_bin_width)
    bin_boundary_low = bin_boundary[:500]
    bin_boundary_upp = bin_boundary[1:]
    return np.sqrt(
        (bin_boundary_low - phys.me)*(bin_boundary_upp - phys.me)
    )    

The transfer function is calculated by injecting 2 photons at the input energy abscissa. However, the first step is to assign $x$ photons to the top bin of the *output* photon abscissa, such that $x$ times the energy of the top bin is 2 times the injection energy. Because of the misalignment between the two abscissae, as well as the fact that we want to transfer function for a single electron, we have to normalize the results first.

First, we construct two arrays: a list of output abscissae (for both photons and electrons) given the injection abscissa, and a list of the energy of the *output* photon energy bin where the injected photons are assigned to. Then we compute the normalization factor. 

In [7]:
# dimensions input x output
out_photeng_arr = np.array([get_out_photeng(eng) for eng in in_eng_arr])
# dimensions input x output
out_eleceng_arr = np.array([get_out_eleceng(eng) for eng in in_eng_arr])

# dimensions input
top_photeng_bin_arr = np.array(
    [
        photeng[photeng < eng][-1] 
        for eng,photeng in zip(in_eng_arr, out_photeng_arr)
    ]
)

# dimensions input
norm_fac_arr = in_eng_arr/top_photeng_bin_arr*2

Dividing by the normalization factor gives the spectra for the injection of 1 photon, with energy given by top_photeng_bin_arr. However, we want the transfer function for a dN/dE = 1 of an array of energy *bins*, with abscissa `in_eng_arr`. To find normalization factor for going from a photon of a certain energy to dN/dE = 1 in an energy bin can be obtained from `spectools.rebin_N_arr`, which takes an array of number of photons and their energies, and puts them into bins.

In [8]:
# norm_fac_arr *= spectools.rebin_N_arr(
#     np.ones_like(top_photeng_bin_arr), 
#     top_photeng_bin_arr
# ).dNdE

# print(norm_fac_arr)

Now we can construct the raw `Spectrum` lists that we will finally put into a `TransFuncAtEnergy` object for the high energy photons, low energy photons and low energy electrons. These are spectra with input energy given by dN/dE = 1 for each bin of `in_eng_arr`. 

In [9]:
# There are some zero values in the raw data files. Position at which they occur seem quite fixed. 
# raw_tf_highengphot = raw_tf[:,:,:,:,0]
# raw_tf_lowengphot = raw_tf[:,:,:,:,1]
# raw_tf_lowengelec = raw_tf[:,:,:,:,2]
# print(np.where(raw_tf_highengphot < 0))
# print(raw_tf[32, 0, 137, 214, 0])
# print(np.log10(in_eng_arr[137]))
# print(np.exp(log_rs_arr[32]))
# print(raw_tf_highengphot[raw_tf_highengphot < 0])
# print(raw_tf_lowengphot[raw_tf_lowengphot < 0])
# print(raw_tf_lowengelec[raw_tf_lowengelec < 0])

In [10]:
photspec_list = [
    [
        Spectrum(
            out_photeng, raw_tf[i,0,j,:,0]/norm_fac, 
            rs = np.exp(log_rs), in_eng = in_eng
        ) 
        for (i, log_rs) in enumerate(log_rs_arr)
    ] for (j, (in_eng, out_photeng, norm_fac)) in enumerate(
            zip(top_photeng_bin_arr, out_photeng_arr, norm_fac_arr)
    )
]

lowengphotspec_list = [
    [
        Spectrum(
            out_photeng, raw_tf[i,0,j,:,1]/norm_fac, 
            rs = np.exp(log_rs), in_eng = in_eng
        ) 
        for (i, log_rs) in enumerate(log_rs_arr)
    ] for (j, (in_eng, out_photeng, norm_fac)) in enumerate(
            zip(in_eng_arr, out_photeng_arr, norm_fac_arr)
    )
]

lowengelecspec_list = [
    [
        Spectrum(
            out_eleceng, raw_tf[i,0,j,:,2]/norm_fac, 
            rs = np.exp(log_rs), in_eng = in_eng
        ) 
        for (i, log_rs) in enumerate(log_rs_arr)
    ] for (j, (in_eng, out_eleceng, norm_fac)) in enumerate(
            zip(in_eng_arr, out_eleceng_arr, norm_fac_arr)
    )
]

Now we rebin all of the `Spectrum` objects. The final abscissa that we would like to use is `get_out_photeng(in_eng_arr[-1])`, i.e. the photon abscissa corresponding to the largest injected energy, as well as `get_out_eleceng(in_eng_arr[-1])` for the electrons. We also change the `Spectrum` object to type `N`.

In [11]:
fin_photeng = get_out_photeng(in_eng_arr[-1])
fin_eleceng = get_out_eleceng(in_eng_arr[-1])

for phot_specs in tqdm(photspec_list):
    for phot_spec in phot_specs:
        phot_spec.rebin_smooth(top_photeng_bin_arr, a=1, b=0)
        phot_spec.switch_spec_type()
        print(phot_spec.spec_type)
#         phot_spec.at_eng(top_photeng_bin_arr)
#         print('$$$$$')
#         phot_spec.rebin(fin_photeng)
        
for phot_specs in tqdm(lowengphotspec_list):
    for phot_spec in phot_specs:
        phot_spec.rebin(fin_photeng)
        phot_spec.switch_spec_type()

for elec_specs in tqdm(lowengelecspec_list):
    for elec_spec in elec_specs:
        elec_spec.rebin(fin_eleceng)
        elec_spec.switch_spec_type()
    

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

N
N
N
N
N


KeyboardInterrupt: 

Finally, we can construct the `TransferFuncList` from this.

In [ ]:
tfunclist_photspec = tflist.TransferFuncList(
    [
        tf.TransFuncAtEnergy(spec_arr, dlnz=0.002)
        for spec_arr in photspec_list
    ]
)

tfunclist_lowengphotspec = tflist.TransferFuncList(
    [
        tf.TransFuncAtEnergy(spec_arr, dlnz=0.002)
        for spec_arr in lowengphotspec_list
    ]
)

tfunclist_lowengelecspec = tflist.TransferFuncList(
    [
        tf.TransFuncAtEnergy(spec_arr, dlnz=0.002)
        for spec_arr in lowengelecspec_list
    ]
)

In [ ]:
pickle.dump(tfunclist_photspec, 
           open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_photspec.raw", "wb")
           )

pickle.dump(tfunclist_lowengphotspec, 
           open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec.raw", "wb")
           )

pickle.dump(tfunclist_lowengelecspec, 
           open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec.raw", "wb")
           )

At the end, we have the normalized transfer functions, which will takes in a `Spectrum` of number of particles in each bin, and outputs the `Spectrum`, also number of particles.